# Analiza Metod XAI dla Rozpoznawania Emocji z Danych Audio

Ten notebook przedstawia kompleksowe porównanie różnych metod wyjaśnialnej sztucznej inteligencji (XAI) zastosowanych do modelu ResNet klasyfikującego emocje na podstawie spektrogramów Mela. Celem projektu jest zbadanie, które obszary spektrogramu są najistotniejsze dla rozpoznawania poszczególnych emocji oraz porównanie skuteczności różnych technik XAI w kontekście danych dźwiękowych.

## Zawartość Notebooka

Notebook zawiera następujące sekcje:

1. **Importy i konfiguracja** - inicjalizacja niezbędnych bibliotek i ustawienie parametrów
2. **Wczytanie modelu i danych** - załadowanie wytrenowanego modelu ResNet i przygotowanie danych testowych
3. **Funkcja predykcji dla metod XAI** - implementacja mechanizmu przewidywania służącego metodom XAI
4. **Generowanie wyjaśnień** - tworzenie wyjaśnień przy użyciu czterech różnych metod XAI
5. **Wizualizacja porównawcza** - wyświetlenie wyjaśnień dla każdej emocji
6. **Weryfikacja wyjaśnień** - obiektywna ocena skuteczności metod XAI
7. **Porównanie wszystkich metod** - analiza statystyczna wydajności metod
8. **Wnioski końcowe** - podsumowanie najlepszych i najgorszych technik

## Zaimplementowane Metody XAI

1. **LIME (Local Interpretable Model-agnostic Explanations)** - metoda tworząca lokalny model zastępczy, który przybliża zachowanie czarnej skrzynki w okolicy konkretnej próbki
2. **LRP (Layer-wise Relevance Propagation)** - technika propagacji wstecznej, która dystrybuuje wkład każdego piksela wejściowego w końcową predykcję
3. **GradCAM (Gradient-weighted Class Activation Mapping)** - metoda wykorzystująca gradienty płynące do ostatniej warstwy konwolucyjnej do generowania map aktywacji
4. **Wygładzone Mapy Ciszy (Smooth Saliency Maps)** - nowatorskie podejście identyfikujące obszary spektrogramu, które najmniej przyczyniają się do klasyfikacji

## Struktura Modułów

Projekt wykorzystuje następujące moduły:

- **xai_methods/** - zawiera implementacje wszystkich metod XAI:
  - `lime_explainer.py` - implementacja LIME dostosowana do spektrogramów
  - `lrp_explainer.py` - implementacja propagacji istotności w warstwach
  - `gradcam_explainer.py` - implementacja GradCAM dla modelu ResNet
  - `silence_maps_explainer.py` - implementacja wygładzonych map ciszy
  - `visualization_utils.py` - narzędzia do wizualizacji wyjaśnień
  
- **helpers/** - moduły pomocnicze:
  - `resnet_model_definition.py` - definicja modelu AudioResNet

- **config.py** - globalna konfiguracja projektu, ścieżki do plików i parametry

## Metodologia Weryfikacji

Każda metoda XAI jest weryfikowana poprzez:
1. Zasłonięcie obszarów zidentyfikowanych jako najbardziej istotne
2. Zasłonięcie losowych obszarów o tej samej wielkości
3. Porównanie spadku prawdopodobieństwa klasyfikacji w obu przypadkach

Skuteczna metoda XAI powinna wykazywać znacznie większy spadek prawdopodobieństwa po zasłonięciu obszarów istotnych w porównaniu do losowych.

## Wymagania

Projekt wymaga następujących bibliotek:
- PyTorch
- NumPy
- Matplotlib
- Pandas
- Seaborn
- LIME
- [opcjonalnie] CUDA dla akceleracji GPU

## Zastosowanie

Notebook demonstruje, jak techniki XAI mogą pomóc zrozumieć, które części spektrogramu są najważniejsze dla rozpoznawania emocji, co może być wykorzystane do:
- Ulepszenia algorytmów rozpoznawania emocji
- Identyfikacji cech dźwięku istotnych dla poszczególnych emocji
- Diagnostyki i debugowania modelu uczenia maszynowego
- Zwiększenia zaufania do systemu rozpoznawania emocji



## 1. Importy i konfiguracja

In [1]:
import sys
import os

# Dodaj katalog główny projektu do sys.path
current_dir = (
    os.path.dirname(os.path.abspath(__file__))
    if "__file__" in globals()
    else os.getcwd()
)
project_root = os.path.abspath(os.path.join(current_dir, "..", ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Katalog główny projektu: {project_root}")
print(f"Czy katalog src istnieje: {os.path.exists(os.path.join(project_root, 'src'))}")

# Standardowe biblioteki
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

# Biblioteki ML
import torch
import torch.nn.functional as F

# Konfiguracja projektu
from src.config import MODEL_DIR

# Import naszych modułów XAI
from src.XAI.xai_methods.lime_explainer import (
    explain_with_lime,
    verify_lime_explanation,
)
from src.XAI.xai_methods.lrp_explainer import LRP, verify_lrp_explanation
from src.XAI.xai_methods.gradcam_explainer import GradCAM, verify_gradcam_explanation
from src.XAI.xai_methods.silence_maps_explainer import (
    create_smooth_silence_map,
    verify_silence_map,
)
from src.XAI.xai_methods.visualization_utils import (
    normalize_sample,
    display_xai_comparison,
)


from src.helpers.resnet_model_definition import AudioResNet

# Konfiguracja wykresów
plt.rcParams["figure.facecolor"] = "black"
plt.rcParams["axes.facecolor"] = "black"
plt.rcParams["axes.labelcolor"] = "white"
plt.rcParams["axes.titlecolor"] = "white"
plt.rcParams["xtick.color"] = "white"
plt.rcParams["ytick.color"] = "white"

# Określenie urządzenia (CPU lub GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Urządzenie: {device}")

Katalog główny projektu: c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition
Czy katalog src istnieje: True
Urządzenie: cpu


## 2. Wczytanie modelu i danych

In [2]:
# Automatyczne wyszukiwanie pliku modelu
model_files = glob.glob(os.path.join(MODEL_DIR, "best_model_*.pt"))
if len(model_files) == 0:
    raise FileNotFoundError(
        f"Nie znaleziono żadnego pliku modelu w katalogu: {MODEL_DIR}"
    )
else:
    MODEL_PATH = model_files[0]  # Wybierz pierwszy znaleziony plik
    print(f"Załadowano model: {MODEL_PATH}")

# Załadowanie modelu
model = AudioResNet()
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()
print("Model załadowany i gotowy do użycia.")

# Struktura modelu
print("\nStruktura modelu AudioResNet:")
for name, module in model.named_modules():
    if name != "":  # Pomijamy moduł główny
        print(f"- {name}: {module.__class__.__name__}")

Załadowano model: c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\ResNet_mel\model_outputs\best_model_20250515_090117.pt
Model załadowany i gotowy do użycia.

Struktura modelu AudioResNet:
- resnet: ResNet
- resnet.conv1: Conv2d
- resnet.bn1: BatchNorm2d
- resnet.relu: ReLU
- resnet.maxpool: MaxPool2d
- resnet.layer1: Sequential
- resnet.layer1.0: BasicBlock
- resnet.layer1.0.conv1: Conv2d
- resnet.layer1.0.bn1: BatchNorm2d
- resnet.layer1.0.relu: ReLU
- resnet.layer1.0.conv2: Conv2d
- resnet.layer1.0.bn2: BatchNorm2d
- resnet.layer1.1: BasicBlock
- resnet.layer1.1.conv1: Conv2d
- resnet.layer1.1.bn1: BatchNorm2d
- resnet.layer1.1.relu: ReLU
- resnet.layer1.1.conv2: Conv2d
- resnet.layer1.1.bn2: BatchNorm2d
- resnet.layer2: Sequential
- resnet.layer2.0: BasicBlock
- resnet.layer2.0.conv1: Conv2d
- resnet.layer2.0.bn1: BatchNorm2d
- resnet.layer2.0.relu: ReLU
- resnet.layer2.0.conv2: Conv2d
- resnet.layer2.0.bn2: BatchNorm2d
- resnet.layer2.0.downsample: Sequentia

In [3]:
# Wczytanie danych testowych
X_test = np.load(os.path.join(MODEL_DIR, "X_test.npy"))
y_test = np.load(os.path.join(MODEL_DIR, "y_test.npy"))

print(f"Kształt danych testowych: {X_test.shape}")
print(f"Liczba etykiet testowych: {len(y_test)}")

# Wczytanie mapowania etykiet na emocje
label_mapping = np.load(
    os.path.join(MODEL_DIR, "label_mapping.npy"), allow_pickle=True
).item()
reverse_label_mapping = {v: k for k, v in label_mapping.items()}  # Odwrócenie mapowania
print(f"\nMapowanie etykiet: {reverse_label_mapping}")

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\kubas\\Desktop\\Projekt dyplomowy\\Audio-Emotion-Recognition\\src\\ResNet_mel\\model_outputs\\X_test.npy'

In [ ]:
# Wybierz 2 przykłady dla każdej emocji do analizy XAI
emotion_samples = {}

# Iteracja przez dane testowe
for i, label in enumerate(y_test):
    emotion = reverse_label_mapping[label]  # Zamiana etykiety na nazwę emocji

    # Jeśli emocja nie jest jeszcze w słowniku, dodaj ją
    if emotion not in emotion_samples:
        emotion_samples[emotion] = []

    # Dodaj próbkę, jeśli liczba przykładów dla tej emocji jest mniejsza niż 2
    if len(emotion_samples[emotion]) < 2:
        emotion_samples[emotion].append(X_test[i])

    # Sprawdź czy mamy już po 2 przykłady dla każdej emocji
    if all(len(samples) >= 2 for samples in emotion_samples.values()):
        break

# Wyświetlenie liczby próbek dla każdej emocji
for emotion, samples in emotion_samples.items():
    print(f"Emocja: {emotion}, Liczba przykładów: {len(samples)}")

## 3. Funkcja predykcji dla metod XAI

In [6]:
def predict_fn(images):
    """
    Funkcja predykcyjna dla metod XAI.
    Przyjmuje obrazy w formacie [N, H, W, C], przekształca je na tensor PyTorch
    i zwraca prawdopodobieństwa dla każdej klasy.
    """
    # Zamiana obrazów na tensor PyTorch i permutacja wymiarów [N, H, W, C] -> [N, C, H, W]
    images = torch.tensor(images).permute(0, 3, 1, 2).float()

    # Upewnij się, że dane mają 1 kanał
    images = images[:, :1, :, :]

    # Przewidywanie za pomocą modelu
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1).cpu().numpy()

    return probabilities

## 4. Generowanie wyjaśnień dla wszystkich metod XAI

In [7]:
# Inicjalizacja obiektów dla metod XAI
gradcam = GradCAM(model, "layer4")
lrp_analyzer = LRP(model)

# Słowniki do przechowywania wyjaśnień
all_explanations = {}

In [ ]:
# Generowanie wyjaśnień dla wszystkich próbek
for emotion, samples in emotion_samples.items():
    all_explanations[emotion] = []

    for i, sample in enumerate(samples):
        print(f"Generowanie wyjaśnień dla {emotion}, przykład {i + 1}/{len(samples)}")

        # Przygotowanie próbki
        if sample.shape[0] == 1:
            sample = sample.squeeze(0)

        # Normalizacja
        sample_norm = normalize_sample(sample)

        # Konwersja do formatu PyTorch
        sample_tensor = (
            torch.tensor(sample).unsqueeze(0).unsqueeze(0).float().to(device)
        )

        # 1. LIME
        print("  - Generowanie wyjaśnienia LIME...")
        lime_temp, lime_mask, segments = explain_with_lime(sample_norm, predict_fn)

        # 2. LRP
        print("  - Generowanie wyjaśnienia LRP...")
        relevance_map = lrp_analyzer.generate_relevance_map(sample_tensor)

        # 3. GradCAM
        print("  - Generowanie wyjaśnienia GradCAM...")
        gradcam_map = gradcam.generate_cam(sample_tensor)

        # 4. Smooth Saliency Maps
        print("  - Generowanie wygładzonej mapy ciszy...")
        silence_map = create_smooth_silence_map(
            sample, n_samples=30, noise_level=0.03, percentile=25
        )

        # Zapisz wszystkie wyjaśnienia
        all_explanations[emotion].append(
            {
                "sample": sample,
                "lime": (lime_temp, lime_mask),
                "lrp": relevance_map,
                "gradcam": gradcam_map,
                "silence": silence_map,
            }
        )

## 5. Wizualizacja porównawcza dla każdej emocji

In [ ]:
# Wizualizacja porównań dla wszystkich emocji
for emotion, explanations in all_explanations.items():
    print(f"\n=== Emocja: {emotion} ===")

    for i, exp in enumerate(explanations):
        sample = exp["sample"]

        # Przygotowanie wyjaśnień do wyświetlenia
        xai_results = {
            "lime": exp["lime"],
            "gradcam": exp["gradcam"],
            "lrp": exp["lrp"],
            "silence": exp["silence"],
        }

        # Użycie funkcji wizualizacyjnej z visualization_utils
        fig = display_xai_comparison(sample, xai_results, emotion)
        plt.show()

## 6. Weryfikacja wyjaśnień

In [ ]:
# Weryfikacja LIME
print("\n=== Weryfikacja LIME ===")
lime_results = []

for emotion, explanations in all_explanations.items():
    for i, exp in enumerate(explanations):
        sample = exp["sample"]
        lime_temp, lime_mask = exp["lime"]

        result = verify_lime_explanation(
            model, sample, lime_mask, device, reverse_label_mapping
        )
        result["emotion"] = emotion
        lime_results.append(result)

        print(f"Emocja: {emotion}, Przykład {i + 1}")
        print(
            f"  - Spadek po zasłonięciu ważnych obszarów: {result['prob_drop_important']:.4f}"
        )
        print(
            f"  - Spadek po zasłonięciu losowych obszarów: {result['prob_drop_random']:.4f}"
        )

lime_df = pd.DataFrame(lime_results)
lime_summary = lime_df.groupby("emotion")[
    ["prob_drop_important", "prob_drop_random"]
].mean()
print("\nŚrednie wyniki dla LIME:")
print(lime_summary)

In [ ]:
# Weryfikacja LRP
print("\n=== Weryfikacja LRP ===")
lrp_results = []

for emotion, explanations in all_explanations.items():
    for i, exp in enumerate(explanations):
        sample = exp["sample"]
        relevance_map = exp["lrp"]

        result = verify_lrp_explanation(
            model, sample, relevance_map, device, reverse_label_mapping
        )
        result["emotion"] = emotion
        lrp_results.append(result)

        print(f"Emocja: {emotion}, Przykład {i + 1}")
        print(
            f"  - Spadek po zasłonięciu ważnych obszarów: {result['prob_drop_important']:.4f}"
        )
        print(
            f"  - Spadek po zasłonięciu losowych obszarów: {result['prob_drop_random']:.4f}"
        )

lrp_df = pd.DataFrame(lrp_results)
lrp_summary = lrp_df.groupby("emotion")[
    ["prob_drop_important", "prob_drop_random"]
].mean()
print("\nŚrednie wyniki dla LRP:")
print(lrp_summary)

In [ ]:
# Weryfikacja GradCAM
print("\n=== Weryfikacja GradCAM ===")
gradcam_results = []

for emotion, explanations in all_explanations.items():
    for i, exp in enumerate(explanations):
        sample = exp["sample"]
        gradcam_map = exp["gradcam"]

        result = verify_gradcam_explanation(
            model, sample, gradcam_map, device, reverse_label_mapping
        )
        result["emotion"] = emotion
        gradcam_results.append(result)

        print(f"Emocja: {emotion}, Przykład {i + 1}")
        print(
            f"  - Spadek po zasłonięciu ważnych obszarów: {result['prob_drop_important']:.4f}"
        )
        print(
            f"  - Spadek po zasłonięciu losowych obszarów: {result['prob_drop_random']:.4f}"
        )

gradcam_df = pd.DataFrame(gradcam_results)
gradcam_summary = gradcam_df.groupby("emotion")[
    ["prob_drop_important", "prob_drop_random"]
].mean()
print("\nŚrednie wyniki dla GradCAM:")
print(gradcam_summary)

In [ ]:
# Weryfikacja Silence Maps
print("\n=== Weryfikacja Smooth Silence Maps ===")
silence_results = []

for emotion, explanations in all_explanations.items():
    for i, exp in enumerate(explanations):
        sample = exp["sample"]
        silence_map = exp["silence"]

        result = verify_silence_map(
            model, sample, silence_map, device, reverse_label_mapping
        )
        result["emotion"] = emotion
        silence_results.append(result)

        print(f"Emocja: {emotion}, Przykład {i + 1}")
        print(
            f"  - Spadek po zasłonięciu obszarów ciszy: {result['prob_drop_silence']:.4f}"
        )
        print(
            f"  - Spadek po zasłonięciu losowych obszarów: {result['prob_drop_random']:.4f}"
        )

silence_df = pd.DataFrame(silence_results)
silence_summary = silence_df.groupby("emotion")[
    ["prob_drop_silence", "prob_drop_random"]
].mean()
print("\nŚrednie wyniki dla Smooth Silence Maps:")
print(silence_summary)

## 7. Porównanie wszystkich metod XAI

In [ ]:
# Tworzenie ramki danych porównawczej dla wszystkich metod
comparison_data = {}

# Zbieranie danych dla różnych metod
for emotion in emotion_samples.keys():
    comparison_data[emotion] = {
        "LIME_important": lime_summary.loc[emotion, "prob_drop_important"]
        if emotion in lime_summary.index
        else np.nan,
        "LIME_random": lime_summary.loc[emotion, "prob_drop_random"]
        if emotion in lime_summary.index
        else np.nan,
        "LRP_important": lrp_summary.loc[emotion, "prob_drop_important"]
        if emotion in lrp_summary.index
        else np.nan,
        "LRP_random": lrp_summary.loc[emotion, "prob_drop_random"]
        if emotion in lrp_summary.index
        else np.nan,
        "GradCAM_important": gradcam_summary.loc[emotion, "prob_drop_important"]
        if emotion in gradcam_summary.index
        else np.nan,
        "GradCAM_random": gradcam_summary.loc[emotion, "prob_drop_random"]
        if emotion in gradcam_summary.index
        else np.nan,
        "Silence_important": silence_summary.loc[emotion, "prob_drop_silence"]
        if emotion in silence_summary.index
        else np.nan,
        "Silence_random": silence_summary.loc[emotion, "prob_drop_random"]
        if emotion in silence_summary.index
        else np.nan,
    }

comparison_df = pd.DataFrame(comparison_data).T

# Wyświetlanie wyników
print("Porównanie wszystkich metod XAI:")
print(comparison_df)

In [ ]:
# Wizualizacja porównawcza spadku prawdopodobieństwa dla ważnych obszarów
important_cols = [
    col
    for col in comparison_df.columns
    if ("important" in col or "silence" in col) and "random" not in col
]
plt.figure(figsize=(14, 8))
comparison_df[important_cols].plot(kind="bar", figsize=(14, 8))
plt.title(
    "Porównanie spadku prawdopodobieństwa dla obszarów istotnych",
    fontsize=16,
    color="white",
)
plt.ylabel("Spadek prawdopodobieństwa", fontsize=14, color="white")
plt.xlabel("Emocje", fontsize=14, color="white")
plt.legend(["LIME", "LRP", "GradCAM", "Smooth Saliency Maps "], labelcolor="white")
plt.xticks(rotation=0, color="white")
plt.tight_layout()
plt.show()

In [ ]:
# Obliczenie różnicy między spadkiem dla ważnych i losowych obszarów (skuteczność metody XAI)
efficiency_df = pd.DataFrame()
efficiency_df["LIME"] = comparison_df["LIME_important"] - comparison_df["LIME_random"]
efficiency_df["LRP"] = comparison_df["LRP_important"] - comparison_df["LRP_random"]
efficiency_df["GradCAM"] = (
    comparison_df["GradCAM_important"] - comparison_df["GradCAM_random"]
)
# Dla Silence Maps patrzymy na przeciwną różnicę, bo chcemy żeby był mniejszy spadek dla obszarów ciszy
efficiency_df["Silence_Maps"] = (
    comparison_df["Silence_random"] - comparison_df["Silence_important"]
)

# Wyświetlanie skuteczności metod
print(
    "Skuteczność metod XAI (różnica między spadkiem dla obszarów istotnych i losowych):"
)
print(efficiency_df)
print("\nŚrednia skuteczność dla wszystkich emocji:")
print(efficiency_df.mean())

# Wizualizacja skuteczności
plt.figure(figsize=(14, 8))
efficiency_df.plot(kind="bar", figsize=(14, 8))
plt.title("Skuteczność metod XAI dla każdej emocji", fontsize=16, color="white")
plt.ylabel("Skuteczność (większa wartość = lepsza metoda)", fontsize=14, color="white")
plt.xlabel("Emocje", fontsize=14, color="white")
plt.legend(["LIME", "LRP", "GradCAM", "Smooth Saliency Maps"], labelcolor="white")
plt.axhline(y=0, color="r", linestyle="-", alpha=0.3)
plt.xticks(rotation=0, color="white")
plt.tight_layout()
plt.show()

## 8. Wnioski i porównanie końcowe

In [ ]:
# Podsumowanie skuteczności metod
best_method = efficiency_df.mean().idxmax()
mean_efficiency = efficiency_df.mean()

print("Podsumowanie skuteczności metod XAI:")
for method, score in mean_efficiency.items():
    status = "✅ Skuteczna" if score > 0 else "❌ Nieskuteczna"
    print(f"{method}: {score:.4f} - {status}")

print(
    f"\nNajlepsza metoda dla tego modelu: {best_method} (średnia skuteczność: {mean_efficiency[best_method]:.4f})"
)